In [38]:
"""Generative adversarial network."""

import numpy as np
import tensorflow as tf

from tensorflow import contrib
from tensorflow.contrib import layers

class Gan(object):
    """Adversary based generator network.
    """
    def __init__(self, ndims=784, nlatent=2):
        """Initializes a GAN

        Args:
            ndims(int): Number of dimensions in the feature.
            nlatent(int): Number of dimensions in the latent space.
        """
        tf.reset_default_graph()
        self._ndims = ndims
        self._nlatent = nlatent

        # Input images
        self.x_placeholder = tf.placeholder(tf.float32, [None, self._ndims])

        # Input noise
        self.z_placeholder = tf.placeholder(tf.float32, [None, self._nlatent])
        
        # Build graph.
        self.x_hat = self._generator(self.z_placeholder)
        y_hat = self._discriminator(self.x_hat)
        y = self._discriminator(self.x_placeholder, reuse=True)
        

        # Discriminator loss
        self.d_loss = self._discriminator_loss(y, y_hat)

        # Generator loss
        self.g_loss = self._generator_loss(y_hat)

        # Add optimizers for appropriate variables
        self.lr_placeholder = tf.placeholder(tf.float32, [])
        
        self.model_param = tf.trainable_variables()
        self.g_param = [p for p in self.model_param if p.name.startswith('generator')]
        self.d_param = [p for p in self.model_param if p.name.startswith('discriminator')]
        
        
        self.g_train_opt = tf.train.AdamOptimizer(learning_rate=self.lr_placeholder)\
                                        .minimize(self.g_loss, var_list=self.g_param)
        self.d_train_opt = tf.train.AdamOptimizer(learning_rate=self.lr_placeholder)\
                                        .minimize(self.d_loss, var_list=self.d_param)

        # Create session
        self.session = tf.InteractiveSession()
        self.session.run(tf.global_variables_initializer())
        


    def _discriminator(self, x, reuse=False):
        """Discriminator block of the network.

        Args:
            x (tf.Tensor): The input tensor of dimension (None, 784).
            reuse (Boolean): re use variables with same name in scope instead of creating
              new ones, check Tensorflow documentation
        Returns:
            y (tf.Tensor): Scalar output prediction D(x) for true vs fake image(None, 1). 
              DO NOT USE AN ACTIVATION FUNCTION AT THE OUTPUT LAYER HERE.

        """
        with tf.variable_scope("discriminator", reuse=reuse) as scope:
            layer_1 = layers.fully_connected(x,256,activation_fn=None)
            layer_1 = tf.maximum(0.01*layer_1, layer_1)
            layer_2 = layers.fully_connected(layer_1,128 , activation_fn=None)
            layer_2 = tf.maximum(0.01*layer_2, layer_2)
            layer_3 = layers.fully_connected(layer_2, 1, activation_fn=None)
            
            return layer_3


    def _discriminator_loss(self, y, y_hat):
        """Loss for the discriminator.

        Args:
            y (tf.Tensor): The output tensor of the discriminator for true images of dimension (None, 1).
            y_hat (tf.Tensor): The output tensor of the discriminator for fake images of dimension (None, 1).
        Returns:
            l (tf.Scalar): average batch loss for the discriminator.

        """
        loss_real = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=y,
                                                    labels=tf.ones_like(y)*0.9))
        loss_fake = tf.reduce_mean(
                  tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, 
                                                          labels=tf.zeros_like(y_hat)))
        l = loss_real + loss_fake
        return l


    def _generator(self, z, reuse=False):
        """From a sampled z, generate an image.

        Args:
            z(tf.Tensor): z from _sample_z of dimension (None, 2).
            reuse (Boolean): re use variables with same name in scope instead of creating
              new ones, check Tensorflow documentation 
        Returns:
            x_hat(tf.Tensor): Fake image G(z) (None, 784).
        """
        with tf.variable_scope("generator", reuse=reuse) as scope:
            layer_1 = layers.fully_connected(z, 128, activation_fn=None)
            layer_1 = tf.maximum(0.01*layer_1, layer_1)
            layer_2 = layers.fully_connected(layer_1, 256, activation_fn=None)
            layer_2 = tf.maximum(0.01*layer_2, layer_2)
            x_hat = layers.fully_connected(layer_2, self._ndims, activation_fn=tf.nn.tanh)
    
            return x_hat


    def _generator_loss(self, y_hat):
        """Loss for the discriminator.

        Args:
            y_hat (tf.Tensor): The output tensor of the discriminator for fake images of dimension (None, 1).
        Returns:
            l (tf.Scalar): average batch loss for the discriminator.

        """
        l = tf.reduce_mean(
             tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat,
                                                     labels=tf.ones_like(y_hat)))
        return l
  

In [39]:
def train(model, mnist_dataset, learning_rate=0.0001, batch_size=32,
          num_steps=5000):
    """Implements the training loop of stochastic gradient descent.

    Performs stochastic gradient descent with the indicated batch_size and
    learning_rate.

    Args:
        model(GAN): Initialized generative network.
        mnist_dataset: input_data.
        learning_rate(float): Learning rate.
        batch_size(int): batch size used for training.
        num_steps(int): Number of steps to run the update ops.
    """
    for step in range(0, num_steps):
        batch_x, _ = mnist_dataset.train.next_batch(batch_size)
        batch_z = np.random.uniform(-1,1,[batch_size,model._nlatent])
        # Train generator and discriminator
        g_feed={model.z_placeholder: batch_z, model.lr_placeholder: learning_rate}
        d_feed={model.x_placeholder: batch_x, model.z_placeholder:batch_z, model.lr_placeholder: learning_rate}
        model.session.run(model.d_train_opt, feed_dict=d_feed)
        model.session.run(model.g_train_opt,feed_dict=g_feed)
        #print(model.session.run(model.d_loss, feed_dict=d_feed ),model.session.run(model.g_loss, feed_dict=g_feed ) )



In [40]:
"""Generative Adversarial Networks
"""

import input_data
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt





In [41]:
"""High level pipeline.

This scripts performs the training for GANs.
"""
# Get dataset.
mnist_dataset = input_data.read_data_sets('MNIST_data', one_hot=True)

    # Build model.
model = Gan(nlatent=4)

# Start training
train(model, mnist_dataset)


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [42]:

std = 1
x_z = np.linspace(-1*std, 1*std, 20)
y_z = np.linspace(-1*std, 1*std, 20)

out = np.empty((28*20, 28*20))
for x_idx, x in enumerate(x_z):
    for y_idx, y in enumerate(y_z):
        z_mu = np.array([np.random.uniform(-1,1,size=4)])
        img = model.session.run(model.x_hat, feed_dict={model.z_placeholder: z_mu})
        out[x_idx*28:(x_idx+1)*28,y_idx*28:(y_idx+1)*28] = img[0].reshape(28, 28)
plt.imsave('generated_img.png', out, cmap="gray")
